In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import sys 
import datetime as dt
import time
import gc
gc.collect()

LESSON_DATA_FOLDER = './data/'
#http://www.imdb.com/interfaces/

In [33]:
from helper.parallel_util import ParallelUtil
from helper.parsers import Utils
from helper.file_mapping import FILE_MAPPINGS

In [3]:
#clusters = ParallelUtil('./data/parallel.config.tsv')      
#clusters.info()

In [ ]:
import read_data as importLib

In [ ]:
importLib.mergeAll()

In [ ]:
'''
    #@dview.parallel(block=True)
    def f(group):
        #print (len(group))

        #group = pandas.DataFrame(group.to_dict('dict'))

        group = group.set_index('tconst')    
        data = {'directors': group.to_json(orient='records')}
        #data = {'directors': group.to_dict('dict')}
        #print (group.index.values , directors)
        return pandas.DataFrame(data=data, index = group.index.values)  



#title_grouped_directors# = 
#title_grouped_directors.reset_index().drop(['level_1'], axis= 1)
#pd.DataFrame(title_grouped_directors).to_json(orient='records')
#title_grouped_directors.reset_index(level='tconst')
'''

In [ ]:
'''
print (time.strftime("%Y-%m-%d %H:%M:%S"))
clusters = PARALLEL_CLUSTERS()
dview = clusters[:]
dview.scatter("scview", title_grouped_directors.apply(f))
#dview['scview']
title_grouped_directors = pd.concat([i for i in dview["scview"]])
print (time.strftime("%Y-%m-%d %H:%M:%S"))
#title_grouped_directors = title_grouped_directors.reset_index().drop(['level_1'], axis= 1)
'''